<a href="https://colab.research.google.com/github/gabrielborja/python_data_analysis/blob/main/online_learning_analytics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Udacity analytics

Data about Udacity courses and programs can be found [here](https://www.udacity.com/courses/all?type=free%20courses)

## Uploading packages and data

In [1]:
#Importing HTTP libraries
from bs4 import BeautifulSoup as bs

In [2]:
#Importing data manipulation libraries
import numpy as np
import pandas as pd

##Retrieving data from URL

In [5]:
#Removing previous version of the file
!rm udacity.txt

In [6]:
#Uploading file from local drive
from google.colab import files
my_text_1 = files.upload()

Saving udacity.txt to udacity.txt


In [7]:
#Extracting html from txt file
html1 = my_text_1['udacity.txt']

In [125]:
#Create a function to retrieve data from given URL
def get_courses(url):
  """Retrieve data from given URL and return a dataframe"""
  
  #Get data and parse with BeautifulSoup
  try:
    soup = bs(url, 'html.parser')
    
    #Save retrieved data to lists
    d1_names = [i.text for i in soup.find_all('h2')]
    d1_schools = [i.text for i in soup.find_all('h3')]
    d1_description = [i.p.text for i in soup.find_all(class_='card_body__1fi66')]
    d1_skills = [i.p.text if i.p != None else None for i in soup.find_all(class_='card_overview__G6gIz')]
    d1_flag = [i.small.text if i.small != None else 'New' for i in soup.find_all(class_='card_container__aBcfz')]
    d1_level = [i.li.text if i.li.text != '' else 'beginner' for i in soup.find_all(class_='card_stats__StvYz')]
    d1_duration = [i for i in soup.find_all(class_='card_stats__StvYz')]
    
    #Save data to dataframe
    df = pd.DataFrame()
    df = df.assign(name=d1_names, school=d1_schools, description=d1_description,
                   skills = d1_skills, flag=d1_flag, level=d1_level,
                   duration= d1_duration)
    return df

  except TypeError:
    print('TypeError')

In [126]:
#Generating a dataframe from url
df1 = get_courses(html1)
df1.head(5)

,name,school,description,skills,flag,level,duration
0,Data Engineer,School of Data Science,Data Engineering is the foundation for the new...,"Data Modeling, Data Pipelines, Data Lakes, Spa...",Popular,intermediate,"[[intermediate], [5 Months], [[<div class=""rat..."
1,Data Scientist,School of Data Science,"Build effective machine learning models, run d...","Machine Learning, Deep Learning, Software Engi...",Popular,advanced,"[[advanced], [4 Months], [[<div class=""rating-..."
2,Programming for Data Science with Python,School of Data Science,Learn the fundamental programming tools for da...,"Python, Numpy & Pandas, SQL, Git & GitHub",Popular,beginner,"[[beginner], [3 Months], [[<div class=""rating-..."
3,Product Manager,School of Product Management,Envision and execute the development of indust...,"Product Strategy, Product Design, Product Deve...",Popular,beginner,"[[beginner], [4 Months], [[<div class=""rating-..."
4,Digital Marketing,School of Business,Gain real-world experience running live campai...,"Digital Marketing, Social Media Marketing, Fac...",Popular,beginner,"[[beginner], [3 Months], [[<div class=""rating-..."


In [70]:
df1[df1['level'].isna()]

,name,school,description,skills,flag,level


In [110]:
df1['duration'][1]

In [61]:
df1[df1['name']=='Machine Learning DevOps Engineer']

,name,school,description,skills,flag
26,Machine Learning DevOps Engineer,School of Artificial Intelligence,"In this program,you will build theDevOps skill...","API development, Automated model scoring, CI...",New


In [ ]:
#Exporting to excel into local disk
from google.colab import files
df1.to_excel('2021_08_07_udacity.xlsx', index=False) #==> Excluding index from file
files.download('2021_08_07_udacity.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>